In [ ]:
import pandas as pd
import seaborn as sns

---
## map metagenomics to unique ID

In [ ]:
metadata = pd.read_csv('/Volumes/PGH-Backup/ibd_data/metadata/hmp2_metadata_2018-08-20.csv')
rnaseq = metadata[metadata['data_type'] == 'host_transcriptomics']
metagenomics = metadata[metadata['data_type'] == 'metagenomics']

In [ ]:
# display(metagenomics)

In [ ]:
mapping = pd.read_csv('/Volumes/PGH-Backup/ibd_data/metadata/matched_rnaseq_mgx_sample_mapping.csv')

mgx = pd.read_csv('/Volumes/PGH-Backup/ibd_data/humann_second_run/ibd_genefamilies_relab_clustered.tsv', sep='\t')

display(mgx)
display(mapping)

In [ ]:
mgx = pd.merge(mgx, metagenomics[['Participant ID', 'week_num', 'External ID']], left_on='sample_id', right_on='External ID', how='left')

display(mgx)

In [ ]:
mgx['unique_id'] = mgx['Participant ID'] + '_' + mgx['week_num'].astype(str)

display(mgx)

In [ ]:
display(mgx_test)

In [ ]:
mgx = mgx.drop(['Participant ID', 'week_num', 'External ID'], axis=1)

In [ ]:
display(mgx)

In [ ]:
# Assuming the column you want to search is called 'column_name'
result = mgx[mgx['unique_id'].str.contains('C3002')]
print(result)


In [ ]:
mgx.to_csv('/Volumes/PGH-Backup/ibd_data/humann_second_run/ibd_genefamilies_relab_clustered_for_rnaseq.tsv', sep='\t', index=False)

---
## Map RNAseq to unique ID

In [ ]:
rna = pd.read_csv('/Volumes/PGH-Backup/ibd_data/rnaseq/tmm_normalized_counts.tsv', sep='\t')
rna = rna.set_index('Gene').T
rna = rna.reset_index().rename(columns={'index': 'sample_id'})
display(rna)

In [ ]:
display(mapping)

In [ ]:
rna_merged = pd.merge(rna, mapping[['Participant ID', 'week_num_metagenomics', 'External ID']], left_on='sample_id', right_on='External ID', how='left')
display(rna_merged)

In [ ]:
rna_merged['unique_id'] = rna_merged['Participant ID'] + '_' + rna_merged['week_num_metagenomics'].astype(str)

In [ ]:
print(rnaseq['External ID'])

In [ ]:
rna_merged = pd.merge(rna_merged, rnaseq[['biopsy_location', 'External ID']], left_on='sample_id', right_on='External ID', how='left')

In [ ]:
display(rna_merged)

In [ ]:
rna_merged['unique_id'] = rna_merged['Participant ID'] + '_' + rna_merged['week_num'].astype(str)

In [ ]:
display(rna_merged)

In [ ]:
rna_merged.to_csv('/Volumes/PGH-Backup/ibd_data/rnaseq/tmm_normalized_counts_for_mgx.tsv', sep='\t', index=False)

---

In [ ]:
merged_mgx = pd.merge(mapping, mgx, left_on='External ID_metagenomics', right_on='sample_id', how='inner')

display(merged_mgx)

In [ ]:
def find_closest_week(participant_id, week_num, metagenomic_df):
    # Filter the metagenomic_df for the same Participant ID
    participant_metagenomic = metagenomic_df[metagenomic_df["Participant ID"] == participant_id]
    
    # If no metagenomic samples for this participant, return None
    if participant_metagenomic.empty:
        return None
    
    # Calculate the difference between RNAseq week_num and metagenomic week_num
    participant_metagenomic["diff"] = participant_metagenomic["week_num"] - week_num

    print(participant_metagenomic['Participan ID', 'diff'])



In [ ]:
merged_df = pd.merge(rnaseq, metagenomics_filt, on='Participant ID', how='left', suffixes=('_rnaseq', '_metagenomics'))

display(merged_df)

In [ ]:
viewing = merged_df[['Participant ID', 'week_num_rnaseq', 'week_num_metagenomics', 'Project_metagenomics']]

display(viewing)

In [ ]:
# Filter for the closest timepoint or earliest if none is before or at the RNA-seq timepoint
def find_closest_timepoint(group):
    # Filter for metagenomic samples that are before or at the RNA-seq timepoint
    valid_samples = group[group['week_num_metagenomics'] <= group['week_num_rnaseq'].iloc[0]]
    
    if not valid_samples.empty:
        # Find the sample with the closest (maximum) timepoint
        closest_sample = valid_samples.loc[valid_samples['week_num_metagenomics'].idxmax()]
    else:
        # If no valid sample exists, return the earliest metagenomic sample
        closest_sample = group.loc[group['week_num_metagenomics'].idxmin()]
    
    return pd.Series([closest_sample['External ID_metagenomics'], closest_sample['week_num_metagenomics']], 
                     index=['External ID_metagenomics', 'week_num_metagenomics'])

In [ ]:
closest_samples = merged_df.groupby(['Participant ID', 'Project_rnaseq']).apply(find_closest_timepoint).reset_index()

In [ ]:
rnaseq = rnaseq.rename(columns={'Project': 'Project_rnaseq'})

In [ ]:
# Merge the closest samples back to the original RNA-seq DataFrame
final_df = pd.merge(rnaseq, closest_samples, on=['Participant ID', 'Project_rnaseq'], how='left')

In [ ]:
final_df['External ID_metagenomics']

In [ ]:
for_viewing = final_df[['Participant ID', 'External ID', 'External ID_metagenomics', 'week_num', 'week_num_metagenomics', 'Project_rnaseq']]

display(for_viewing)

In [ ]:
for_viewing.to_csv('/Volumes/PGH-Backup/ibd_data/metadata/matched_rnaseq_mgx_sample_mapping.csv', index=False)

**Use Mapping File to Join MGX and RNA Data for Appropriate Samples**

In [ ]:
# Load datasets
mgx = pd.read_csv('/Volumes/PGH-Backup/ibd_data/humann_second_run/ibd_genefamilies_relab_clustered.tsv', sep='\t')
rna = pd.read_csv('/Volumes/PGH-Backup/ibd_data/rnaseq/tmm_normalized_counts.tsv', sep='\t')
mapping = pd.read_csv('/Volumes/PGH-Backup/ibd_data/metadata/matched_rnaseq_mgx_sample_mapping.csv')
metadata = pd.read_csv('/Volumes/PGH-Backup/ibd_data/metadata/hmp2_metadata_2018-08-20.csv')

# windows
# mgx = pd.read_csv('E:/ibd_data/humann_second_run/ibd_genefamilies_relab_clustered.tsv', sep='\t')
# rna = pd.read_csv('E:/ibd_data/rnaseq/tmm_normalized_counts.tsv', sep='\t')
# mapping = pd.read_csv('E:/ibd_data/metadata/matched_rnaseq_mgx_sample_mapping.csv')
# metadata = pd.read_csv('E:/ibd_data/metadata/hmp2_metadata_2018-08-20.csv')


In [ ]:
display(mapping)

In [ ]:
# Tanspose so that Genes are columns and samples are rows in RNAseq data
rna = rna.set_index('Gene').T

rna.shape

In [ ]:
metadata_rna = metadata[metadata['data_type'] == 'host_transcriptomics']

display(metadata_rna)

In [ ]:
display(mgx)
display(rna)
display(mapping)


In [ ]:
rna = rna.reset_index().rename(columns={'index': 'sample_id'})
display(rna.head())

In [22]:
# Create list of enriched NOD2 genes

mac_path = '/Volumes/PGH-Backup/ibd_data/rnaseq/GSE22611_NOD2_VS_CTRL_TRANSDUCED_HEK293T_CELL_UP.v2024.1.Hs.grp'

windows_path = 'E:/ibd_data/rnaseq/GSE22611_NOD2_VS_CTRL_TRANSDUCED_HEK293T_CELL_UP.v2024.1.Hs.grp'


with open (mac_path, 'r') as f:
    gene_list = [line.strip() for line in f.readlines()]
    gene_list = gene_list[2:]

print(gene_list)

['ABCB1', 'ABHD17A', 'ACAD8', 'ACKR4', 'ACTG1', 'AGT', 'AIMP1', 'AK4', 'AMT', 'ANKRD27', 'APBB2', 'ARFGEF1', 'ARHGAP11A', 'ARHGAP6', 'ARHGEF11', 'ATG14', 'ATP13A3', 'ATP2B1', 'B3GNT2', 'BMERB1', 'BMS1', 'BNC2', 'BRD9', 'BSDC1', 'BTG3', 'C1GALT1C1', 'CAPG', 'CD40', 'CD48', 'CD68', 'CD99', 'CDC25B', 'CDK5RAP1', 'CEACAM6', 'CELA3B', 'CENPE', 'CHAF1B', 'CIR1', 'CLCC1', 'CLDN1', 'CNPY3', 'COLGALT1', 'COPS7A', 'CRYZ', 'CSK', 'CUL4A', 'CYB5B', 'DCTN3', 'DCXR', 'DDX46', 'DEGS1', 'DIXDC1', 'DOK2', 'DPAGT1', 'DRICH1', 'DUSP10', 'EBI3', 'EDAR', 'EGLN1', 'EIF2B2', 'EIF5', 'EIF5A', 'ENPP3', 'EXT2', 'FAM114A1', 'FAM193A', 'FAM32A', 'FASTKD5', 'FUT5', 'FXYD6', 'GALNT7', 'GCA', 'GDAP2', 'GLB1', 'GMFG', 'GNPDA1', 'GTF2E1', 'H1-1', 'HADHA', 'HECTD4', 'HIKESHI', 'HLA-DMA', 'HMGB1', 'HNRNPM', 'HTR3B', 'IFNA2', 'IGF1R', 'IGLV1-44', 'IL36G', 'IMPA1', 'INSIG1', 'ITPR1', 'KDM3B', 'KIF22', 'KIF3C', 'KMT5B', 'KPNA2', 'LARS2', 'LCMT1', 'LHX3', 'LILRA1', 'LIMD2', 'LZTR1', 'MAFF', 'MAN1A1', 'MARCHF5', 'MBD3', 'MBD

In [ ]:
# Subset for genes present in table
present_genes = [gene for gene in gene_list if gene in rna.columns]

present_genes.append('sample_id')

In [ ]:
# Subset rnaseq data
rna_filt = rna[present_genes]

display(rna_filt)

In [ ]:
rna_filt = pd.merge(rna, metadata_rna[['External ID', 'biopsy_location']], left_on='sample_id', right_on='External ID', how='left')

display(rna_filt)

In [ ]:
merged_rna = pd.merge(rna_filt, mapping, left_on='sample_id', right_on='External ID', how='inner')

display(merged_rna)

In [ ]:
merged_rna.rename(columns={'week_num': 'week_num_rna', 'week_num_metagenomics': 'week_num'}, inplace=True)

In [ ]:
rna_rectum = merged_rna[merged_rna['biopsy_location'] == 'Rectum']

rna_signmoid_colon = merged_rna[merged_rna['biopsy_location'] == 'Sigmoid Colon']

rna_ileum = merged_rna[merged_rna['biopsy_location'] == 'Ileum']

display(rna_rectum)
display(rna_signmoid_colon)
display(rna_ileum)

In [ ]:
display(mgx)

In [ ]:
metadata_mgx = metadata[metadata['data_type'] == 'metagenomics']

display(metadata_mgx)

In [ ]:
mapped_mgx = pd.merge(mgx, metadata_mgx[['External ID', 'Participant ID', 'week_num']], left_on='sample_id', right_on='External ID', how='left')

display(mapped_mgx)

In [ ]:
merged_rna_mgx_rectum = pd.merge(rna_rectum, mapped_mgx, on=['Participant ID', 'week_num'], how='inner')

display(merged_rna_mgx_rectum)

In [ ]:
merged_rna_mgx_colon = pd.merge(rna_signmoid_colon, mapped_mgx, on=['Participant ID', 'week_num'], how='inner')
display(merged_rna_mgx_colon)

In [ ]:
merged_rna_mgx_ileum = pd.merge(rna_ileum, mapped_mgx, on=['Participant ID', 'week_num'], how='inner')
display(merged_rna_mgx_ileum)

In [ ]:
print(merged_rna_mgx_rectum[['Participant ID', 'week_num']])